### Case Study Groupe 10
Arne Herlinghaus, Max Lütkemeyer, Thomas Mogos, Tim Strauss, Simon Luttmann

# Final Model creation Workflow


#### Imports

In [9]:
import random
from pathlib import Path

import numpy as np, pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve

np.random.seed(42)
random.seed(42)


#### Load original data

In [10]:
train_csv = "data_6_channels_train.csv"
df = (pd.read_csv(train_csv)
      .rename(columns={"numerical_id": "forest_id",
                       "class": "is_disturbance",
                       "BLU": "blue", "GRN": "green", "RED": "red",
                       "NIR": "near_infrared",
                       "SW1": "shortwave_infrared_1", "SW2": "shortwave_infrared_2"}))

#### Feature engineering

In [11]:
def engineer_features(group: pd.DataFrame, lags: int = 3) -> pd.DataFrame:
    eps = 1e-6
    g = group.copy()

    # ---------- Base-Indices ----------------------------------------
    g["NDVI"] = (g.near_infrared - g.red) / (g.near_infrared + g.red + eps)
    g["NDMI"] = (g.near_infrared - g.shortwave_infrared_1) / (g.near_infrared + g.shortwave_infrared_1 + eps)
    g["NDWI"] = (g.green - g.near_infrared) / (g.green + g.near_infrared + eps)
    g["NBR"] = (g.near_infrared - g.shortwave_infrared_2) / (g.near_infrared + g.shortwave_infrared_2 + eps)
    g["EVI"] = 2.5 * (g.near_infrared - g.red) / (g.near_infrared + 6 * g.red - 7.5 * g.blue + 1 + eps)
    g["NBR2"] = (g.shortwave_infrared_1 - g.shortwave_infrared_2) / (
                g.shortwave_infrared_1 + g.shortwave_infrared_2 + eps)
    
    # ---------- Dryness-Ratio -----------------------------------
    g["SWIR_ratio"] = g.shortwave_infrared_2 / (g.shortwave_infrared_1 + eps)

    # ---------- Landsat-8 Tasseled-Cap (Baig 2014) -------------------
    b2, b3, b4 = g.blue, g.green, g.red
    b5, b6, b7 = g.near_infrared, g.shortwave_infrared_1, g.shortwave_infrared_2
    g["TCB"] = 0.3029 * b2 + 0.2786 * b3 + 0.4733 * b4 + 0.5599 * b5 + 0.5080 * b6 + 0.1872 * b7
    g["TCG"] = -0.2941 * b2 - 0.2430 * b3 - 0.5424 * b4 + 0.7276 * b5 + 0.0713 * b6 - 0.1608 * b7
    g["TCW"] = 0.1511 * b2 + 0.1973 * b3 + 0.3283 * b4 + 0.3407 * b5 - 0.7117 * b6 - 0.4559 * b7
    g["TCT4"] = -0.8239 * b2 + 0.0849 * b3 + 0.4396 * b4 - 0.0580 * b5 + 0.2013 * b6 - 0.2773 * b7
    g["TCT5"] = -0.3294 * b2 + 0.0557 * b3 + 0.1056 * b4 + 0.1855 * b5 - 0.4349 * b6 + 0.8085 * b7
    g["TCT6"] = 0.1079 * b2 - 0.9023 * b3 + 0.4119 * b4 + 0.0575 * b5 - 0.0259 * b6 + 0.0252 * b7

    # --- Interaction-Features --------------------------------------
    g["TCBxTCG"] = g.TCB * g.TCG
    g["TCBxTCW"] = g.TCB * g.TCW
    g["TCBxNBR"] = g.TCB * g.NBR

    # ---------- 3-Year-Median, Std, Anomalies (NDVI & NBR) ----------
    for idx in ("NDVI", "NBR"):
        g[f"{idx}_med3"] = g[idx].rolling(3, min_periods=2).median()
        g[f"{idx}_std3"] = g[idx].rolling(3, min_periods=2).std()
        g[f"{idx}_anom"] = (g[idx] - g[f"{idx}_med3"]) / (g[f"{idx}_med3"].abs() + eps)

    # ---------- Lags & Deltas ---------------
    base_cols = [
        "NDVI", "NDMI", "NDWI", "NBR", "EVI", "NBR2", "SWIR_ratio",
        "TCB", "TCG", "TCW", "TCT4", "TCT5", "TCT6",
        "TCBxTCG", "TCBxTCW", "TCBxNBR",
        "blue", "green", "red", "near_infrared", "shortwave_infrared_1", "shortwave_infrared_2"
    ]

    lag_features = []
    for lag in range(1, lags + 1):
        shifted = g[base_cols].shift(lag).rename(columns=lambda col: f"l{lag}_{col}")
        deltas = (g[base_cols] - g[base_cols].shift(lag)).rename(columns=lambda col: f"d{lag}_{col}")
        lag_features.extend([shifted, deltas])
    g = pd.concat([g] + lag_features, axis=1)

    # ---------- Time-Period-Features (assumption: forrest_disturbances occur in periods) ---------------
    g["year_sin"] = np.sin(2 * np.pi * g.year / 10)
    g["year_cos"] = np.cos(2 * np.pi * g.year / 10)

    return g

In [12]:
ft_file = "all_data_all_features.csv"

if not Path(ft_file).exists():
    feats = (df.groupby("forest_id")
             .apply(engineer_features)
             .fillna(method="ffill").fillna(method="bfill")
             .reset_index(drop=True))
    feats.to_csv(ft_file, index=False)
else:
    feats = pd.read_csv(ft_file)

feature_cols = [c for c in feats.columns
                if c not in ("forest_id", "year", "is_disturbance")]


#### Test|Validation 80|20 Split with Groups by forest_id

In [13]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
tr_idx, va_idx = next(gss.split(feats, groups=feats["forest_id"]))

train_df, val_df = feats.iloc[tr_idx], feats.iloc[va_idx]
X_train, y_train = train_df[feature_cols].values, train_df["is_disturbance"].values
X_val, y_val = val_df[feature_cols].values, val_df["is_disturbance"].values
print(f"Train {len(train_df):,} | Val {len(val_df):,}")

Train 362,939 | Val 90,721


 #### Hyperparameter-Tuning XGBoost Model via GridSearch and GroupKFold

In [29]:
np.random.seed(42)

param_grid = {
    "n_estimators":       [1250, 1500, 1750],
    "max_depth":          [5, 6],
    "learning_rate":      [0.02, 0.03, 0.05],
    "subsample":          [0.6, 0.7],
    "colsample_bytree":   [0.7, 0.8],
    "scale_pos_weight":   [3],
    "gamma":              [0.2, 0.25, 0.3],
    "min_child_weight":   [2],
}

search_model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="aucpr",
    eta=0.05,
    subsample=0.8,
    colsample_bytree=0.7,
    min_child_weight=2,
    tree_method="hist",
    random_state=42,
    early_stopping_rounds=60
)

group_kfold = GroupKFold(n_splits=3)
grid_search = GridSearchCV(
    estimator=search_model,
    param_grid=param_grid,
    scoring='f1',
    cv=group_kfold,
    verbose=1,
    n_jobs=-1
)

# Perform the grid search
grid_search.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False, groups=train_df['forest_id'])

# Get the best parameters and the corresponding F1 score
best_params = grid_search.best_params_
best_f1_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best F1 Score: {best_f1_score}")
y_pred = grid_search.best_estimator_.predict(X_val)

# Calculate F1 score and confusion matrix
f1 = f1_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)

print(f"F1 Score on Validation Set: {f1}")
print(f"Confusion Matrix on Validation Set:\n{cm}")

Fitting 3 folds for each of 216 candidates, totalling 648 fits


/Users/tim.strauss/Library/Python/3.9/lib/python/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Parameters: {'colsample_bytree': 0.8, 'gamma': 0.3, 'learning_rate': 0.02, 'max_depth': 6, 'min_child_weight': 2, 'n_estimators': 1250, 'scale_pos_weight': 3, 'subsample': 0.6}
Best F1 Score: 0.611825247337114
F1 Score on Validation Set: 0.6013245033112583
Confusion Matrix on Validation Set:
[[90193   143]
 [  158   227]]


#### Threshold-Optimization

In [39]:
best_model = grid_search.best_estimator_
val_proba = best_model.predict_proba(X_val)[:, 1]
prec, rec, thr = precision_recall_curve(y_val, val_proba)
f1_scores    = 2 * prec * rec / (prec + rec + 1e-9)
best_threshold = float(thr[np.argmax(f1_scores)])
print(f"best threshold (max F1 auf Val): {best_threshold:.3f}")
y_pred = (val_proba > best_threshold).astype(int)
f1 = f1_score(y_val, y_pred)
print (f"F1 Score on Validation Set: {f1:.4f}")

best threshold (max F1 auf Val): 0.515
F1 Score on Validation Set: 0.6016


We chose the XGBoost model because it made the best predictions on our validation set compared to other Models like RandomForest, ExtraTrees, Logistic Regression and K-Nearest Neighbors.

#### Prediction on test dataset

In [40]:
test_csv = "data_6_channels_test_pub.csv"
out_npy = "data_6_channels_test_pub_with_predictions.npy"

orig_test = pd.read_csv(test_csv)

test_tmp = (
    orig_test
    .rename(columns={
        "numerical_id": "forest_id",
        "BLU": "blue",
        "GRN": "green",
        "RED": "red",
        "NIR": "near_infrared",
        "SW1": "shortwave_infrared_1",
        "SW2": "shortwave_infrared_2",
    })
)

test_feat = (
    test_tmp
    .groupby("forest_id", group_keys=False)
    .apply(engineer_features)
    .fillna(method="ffill").fillna(method="bfill")
    .reset_index(drop=True)
)

X_test    = test_feat[feature_cols].values
test_prob = best_model.predict_proba(X_test)[:, 1]
test_pred = (test_prob >= best_threshold).astype(int)

pred_df = (
    test_feat[["forest_id", "year"]]
    .assign(is_disturbance=pred)
    .rename(columns={"forest_id": "numerical_id"})
)

merged = orig_test.merge(
    pred_df,
    on=["numerical_id", "year"],
    how="left"
)

col_order = [
    "is_disturbance"
]
merged = merged[col_order]

np.save(out_npy, merged["is_disturbance"].to_numpy())
print(
    f"✓ '{out_npy}' geschrieben – 0={(merged.is_disturbance == 0).sum():,} | "
    f"1={(merged.is_disturbance == 1).sum():,}"
)


/var/folders/9l/s6j5yq4s12j6wvms4084h50h0000gn/T/ipykernel_93774/2062413000.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_tmp
/var/folders/9l/s6j5yq4s12j6wvms4084h50h0000gn/T/ipykernel_93774/2062413000.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_tmp


✓ 'data_6_channels_test_pub_with_predictions.npy' geschrieben – 0=112,980 | 1=444


In [41]:
pred_loaded = np.load("data_6_channels_test_pub_with_predictions.npy")

if pred_loaded.shape == (113424,): print('Shape is correct')
else: print('Shape is incorrect')

Shape is correct
